In [1]:
##Robs summariser of RC8 regoinal Contributor tables
import pandas as pd
import numpy as np
import os

#mainPath = 'P:\projects\RC9_ResultsSets\RegContribTables'
mainPath = r'\\athenasmb\pdreef\RC8_RC2016\RC8_RC2016_ResultsSets_PointOfTruth'
outPath = r'P:\projects\RC8_ResultsSets'
summariesOutFolder = 'SummariesWithProcessRC10regions'
baseFolderEnd = 'BASE_RC8'
changeFolderEnd = 'CHANGE_RC8'
predevFolderEnd = 'PREDEV_RC8'
regContributorFileName = 'RegContributorDataGrid.csv'
#outputCSVEnd = 'RegToOutlet_Alt.csv'
outputCSVEnd = 'RegToOutlet.csv'
baselineScenarioName = 'Baseline'
changeScenarioName = 'Change'
predevScenarioName = 'PreDev'
reportCardString = 'RC2016'

pathToContstitsToGrpFile = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\constituentsToGroups.csv'

#Must be utilised in regContributors that do NOT include nesting structures
#alternativeRegionsCSV = {'FI':r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\FI_ReportingRegions_Details.csv',
#                        'BU':r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\BU_ReportingRegions_Details.csv'}
alternativeRegionsCSV = {'BU':r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\BU_ReportingRegions_Details.csv'}

altRegLinker = 'Catchmt'
standardRegName = 'RepReg'
altRegColName = 'AUSGMCAS'
#standardRegName = 'RC8P2R'
#altRegColName = 'RC8AUSG'
origLinker = 'ModelElement'


annLoadToStream = 'AnnLoadToStreamKG'
annLoadToExport = 'AnnLoadToExportKG'

regionIDs = ['BU', 'BM', 'CY', 'FI', 'MW', 'WT']
#regionIDs = ['MW']
regionIDs = ['BU']

scenarioToFolderDict = {baselineScenarioName:baseFolderEnd, changeScenarioName:changeFolderEnd, predevScenarioName:predevFolderEnd}


In [2]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

In [3]:
if not os.path.exists(outPath + '\\' + summariesOutFolder):
    os.makedirs(outPath + '\\' + summariesOutFolder)
    print("Made dir: " + outPath + '\\' + summariesOutFolder)

Made dir: P:\projects\RC8_ResultsSets\SummariesWithProcessRC10regions


In [4]:
def getPathInfo(regionIDString, scenarioString):
    fileIn = mainPath + '\\' + regionIDString + '\\Model_Outputs\\' + regionIDString + '_' + scenarioToFolderDict[scenarioString] + '\\' + regContributorFileName
    fileOut = outPath + '\\' + summariesOutFolder + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    return fileIn, fileOut

In [5]:
def produceSummaryFromRegContrib(regionIDString, scenarioName):
    
    theFileIn, theFileOut = getPathInfo(regionIDString, scenarioName)
    
    print("Processing " + theFileIn + " Scenario: " + scenarioName)
    
    rawcontrib = pd.read_csv(theFileIn)
    
    rawcontrib['SCENARIO'] = scenarioName
    rawcontrib[annLoadToStream] = rawcontrib['LoadToStream (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
    rawcontrib[annLoadToExport] = rawcontrib['LoadToRegExport (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
    rawcontrib['AreaHA'] = rawcontrib['AreaM2'].div(10000)

    if regionIDString in alternativeRegionsCSV:
        print("Reading alternative regionalistaion for " + regionIDString)
        altDetsDF = pd.read_csv(alternativeRegionsCSV[regionIDString])
        combinedDF = pd.merge(rawcontrib, altDetsDF, how='left', left_on=[origLinker], right_on = [altRegLinker])
        #Transfer vals
        combinedDF['Rep_Region'] = combinedDF[altRegColName]
        #Drop cols
        #where 1 is the axis number (0 for rows and 1 for columns.)
        combinedDF = combinedDF.drop(altRegColName, 1)
        combinedDF = combinedDF.drop(standardRegName, 1)
        combinedDF = combinedDF.drop(altRegLinker, 1)
        rawcontrib = combinedDF
 
    ### Can keep reporting region in the area table, as this is our Non-Nested notebook
    catchFuArea = pd.DataFrame(rawcontrib.groupby(['Rep_Region','ModelElement','FU']).agg({'AreaHA':'first'})).reset_index()
    regFuArea = pd.DataFrame(catchFuArea.groupby(['Rep_Region','FU']).agg({'AreaHA':'sum'})).reset_index()
    
    #regLuseSummary = pd.DataFrame(rawcontrib[rawcontrib['Constituent'].isin(selectedConstituents)].groupby(['Rep_Region','Constituent','FU','Process']).agg({'AnnLoadToStreamKG':'sum','AnnLoadToExportKG':'sum'})).reset_index()
    regLuseSummary = pd.DataFrame(rawcontrib.groupby(['SCENARIO','Rep_Region','Constituent','FU','Process']).agg({annLoadToStream:'sum',annLoadToExport:'sum'})).reset_index()
    
    ### Join/merge with Regional Areas
    regLuseSumPlusArea = pd.merge(regLuseSummary, regFuArea, how='left', left_on=['Rep_Region','FU'], right_on = ['Rep_Region','FU'])
    
    regLuseSumPlusArea.to_csv(theFileOut, index=False)
    
    print("Saved " + theFileOut)
    
    return

    

In [6]:
#List out the processes

for theReg in regionIDs:
    
    for scenario in scenarioToFolderDict.keys():
        
        print("Doing: " + theReg + " " + scenario)
        produceSummaryFromRegContrib(theReg, scenario)



print("Finished first summary")

Doing: BU Baseline
Processing \\athenasmb\pdreef\RC8_RC2016\RC8_RC2016_ResultsSets_PointOfTruth\BU\Model_Outputs\BU_BASE_RC8\RegContributorDataGrid.csv Scenario: Baseline
Reading alternative regionalistaion for BU
Saved P:\projects\RC8_ResultsSets\SummariesWithProcessRC10regions\BU_RC2016_Baseline_RegToOutlet.csv
Doing: BU Change
Processing \\athenasmb\pdreef\RC8_RC2016\RC8_RC2016_ResultsSets_PointOfTruth\BU\Model_Outputs\BU_CHANGE_RC8\RegContributorDataGrid.csv Scenario: Change
Reading alternative regionalistaion for BU
Saved P:\projects\RC8_ResultsSets\SummariesWithProcessRC10regions\BU_RC2016_Change_RegToOutlet.csv
Doing: BU PreDev
Processing \\athenasmb\pdreef\RC8_RC2016\RC8_RC2016_ResultsSets_PointOfTruth\BU\Model_Outputs\BU_PREDEV_RC8\RegContributorDataGrid.csv Scenario: PreDev
Reading alternative regionalistaion for BU
Saved P:\projects\RC8_ResultsSets\SummariesWithProcessRC10regions\BU_RC2016_PreDev_RegToOutlet.csv
Finished first summary


In [7]:
#Build relationships of constituent names to groups
#tssGrp = 'TSS'
#tnGrp = 'TN'
#tpGrp = 'TP'
#ps2Grp = 'PSII'
#nonps2Grp = 'non-PSII'
##Nope, will read in a CSV to DataFrame instead, thus joing will 'drop' flow from this comparison
#constToGrp = {'Sediment - Fine':tssGrp}

constToGrp = pd.read_csv(pathToContstitsToGrpFile)

baseSupplyStr = 'Base_Supply_KG'
baseExportStr = 'Base_Export_KG'
changeSupplyStr = 'Change_Supply_KG'
changeExportStr = 'Change_Export_KG'
predevSupplyStr = 'PreDev_Supply_KG'
predevExportStr = 'PreDev_Export_KG'
repRegStr = 'Rep_Region'
sourceRegStr = 'Source Region'
constituentStr = 'Constituent'
fuStr = 'FU'
groupStr = 'GROUP'
modelRegStr = 'MODELREG'
percRedStr = 'PercReduction'
regTotalStr = 'REGIONAL TOTAL'
processStr = 'Process'

#allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr])
#allExportsAndReductions = pd.DataFrame()


In [9]:
#So now we would be reading in known summary files

def calculateReductions(regName, thisRegID):
    
    unneededFile, theBaseSummaryFile = getPathInfo(thisRegID, baselineScenarioName)
    unneededFile, theChangeSummaryFile = getPathInfo(thisRegID, changeScenarioName)
    unneededFile, thePreDevSummaryFile = getPathInfo(thisRegID, predevScenarioName)
    
    #print("Reading: " + theBaseSummaryFile)
    baseSummaryDF = pd.read_csv(theBaseSummaryFile)
    #print("Reading: " + theChangeSummaryFile)
    changeSummaryDF = pd.read_csv(theChangeSummaryFile)
    #print("Reading: " + thePreDevSummaryFile)
    predevSummaryDF = pd.read_csv(thePreDevSummaryFile)
    
    #inner drops out those that don't merge, left keep them
    baseTotals = pd.merge(baseSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum', 'AreaHA':'first'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    baseTotals = baseTotals.rename(columns={repRegStr:sourceRegStr,'AreaHA':'AREA_HA',annLoadToStream:baseSupplyStr, annLoadToExport:baseExportStr})
    baseTotals = baseTotals[[sourceRegStr, groupStr, constituentStr, fuStr, processStr, 'AREA_HA', baseSupplyStr, baseExportStr]]
    #baseTotals.head(40)
    
    predevTotals = pd.merge(predevSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    predevTotals = predevTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:predevSupplyStr, annLoadToExport:predevExportStr})
    predevTotals = predevTotals[[sourceRegStr, constituentStr, fuStr, processStr, predevSupplyStr, predevExportStr]]
    
    changeTotals = pd.merge(changeSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT'])
    changeTotals = changeTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:changeSupplyStr, annLoadToExport:changeExportStr})
    changeTotals = changeTotals[[sourceRegStr, constituentStr, fuStr, processStr, changeSupplyStr, changeExportStr]]
    #changeTotals.head(8)
    
    #, columns=['Rep_Region', 'GROUP', 'Constituent', 'FU', 'AREA_HA', 'Base_Supply_KG', 'Base_Export_KG', 'PreDev_Supply_KG', 'PreDev_Export_KG', 'Change_Supply_KG', 'Change_Export_KG']
    anthroRegByLuse = pd.merge(pd.merge(baseTotals, predevTotals, how='left', left_on=[sourceRegStr,constituentStr,fuStr,processStr],
                                        right_on = [sourceRegStr,constituentStr,fuStr,processStr]).reset_index(), changeTotals, how='left',
                               left_on=[sourceRegStr,constituentStr,fuStr,processStr],
                               right_on = [sourceRegStr,constituentStr,fuStr,processStr])
    #anthroRegByLuse.head(8)
    
    theFileOut = outPath + '\\' + summariesOutFolder + '\\' + thisRegID + '_RC2016_AnthroRegByLanduseProcess.csv'
    anthroRegByLuse.to_csv(theFileOut, index=False)
    #print("Saved " + theFileOut)
    
    regExports = anthroRegByLuse.groupby([sourceRegStr, constituentStr, groupStr]).agg({predevExportStr:'sum', baseExportStr:'sum', changeExportStr:'sum'})
    
    regExports[modelRegStr] = regName
    regExports[percRedStr] = (regExports[baseExportStr] - regExports[changeExportStr]).div(regExports[baseExportStr] - regExports[predevExportStr]).mul(100)
    regExports = regExports.reset_index()
    regExports = regExports[[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr]]
    #regExports.head(15)
    
    modelRegExports = regExports.groupby([modelRegStr, constituentStr, groupStr]).agg({predevExportStr:'sum', baseExportStr:'sum', changeExportStr:'sum'})
    modelRegExports[sourceRegStr] = regTotalStr
    modelRegExports[percRedStr] = (modelRegExports[baseExportStr] - modelRegExports[changeExportStr]).div(modelRegExports[baseExportStr] - modelRegExports[predevExportStr]).mul(100)
    modelRegExports = modelRegExports.reset_index()
    modelRegExports = modelRegExports[[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr]]
    ##modelRegExports.head(15)
    regExports = regExports.append(modelRegExports, ignore_index=True)
    
    return regExports


In [10]:
allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr])

theseExports = calculateReductions('Burdekin', 'BU')
allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Burnett Mary', 'BM')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Cape York', 'CY')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Fitzroy', 'FI')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Mackay Whitsunday', 'MW')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Wet Tropics', 'WT')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

theFileNameOut = outPath + '\\' + summariesOutFolder + '\\AllExportsAndReductions_FromProcess.csv'
allExportsAndReductions.to_csv(theFileNameOut, index=False)
allExportsAndReductions.head(20)
#print(allExportsAndReductions)

,MODELREG,Source Region,Constituent,GROUP,PreDev_Export_KG,Base_Export_KG,Change_Export_KG,PercReduction
0,Burdekin,Belyando,Atrazine,PSII,0.000000e+00,2.636022e+00,2.636022e+00,0.000002
1,Burdekin,Belyando,Diuron,PSII,0.000000e+00,0.000000e+00,0.000000e+00,NaN
2,Burdekin,Belyando,Metribuzin,non-priority-PSII,0.000000e+00,0.000000e+00,0.000000e+00,NaN
3,Burdekin,Belyando,N_DIN,TN,6.032863e+04,6.103032e+04,6.103164e+04,-0.188288
4,Burdekin,Belyando,N_DON,TN,1.206573e+05,1.211365e+05,1.211365e+05,0.000000
5,Burdekin,Belyando,N_Particulate,TN,8.465621e+03,5.948945e+04,5.929803e+04,0.375145
6,Burdekin,Belyando,P_DOP,TP,6.032863e+03,6.085714e+03,6.085714e+03,0.000000
7,Burdekin,Belyando,P_FRP,TP,1.809859e+04,1.833554e+04,1.833554e+04,0.000000
8,Burdekin,Belyando,P_Particulate,TP,4.967892e+03,3.763919e+04,3.752954e+04,0.335620
9,Burdekin,Belyando,S-metolachlor,non-PSII,0.000000e+00,0.000000e+00,0.000000e+00,NaN
